In [82]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option("max_colwidth", None)
pd.set_option("max_seq_items", None)
pd.set_option('display.float_format', '{:.2f}'.format)

import certifi
import geopy.geocoders
from geopy.geocoders import Nominatim
import ssl
import time

In [83]:
path = '/Users/tristangarcia/desktop/hp-pred/data/wa/'

In [84]:
train = pd.read_csv(f'{path}wa_train.csv')
test = pd.read_csv(f'{path}wa_validation.csv')  # validation set

In [85]:
print(train.shape)
print(test.shape)

(132522, 14)
(28398, 14)


# Filter

In [86]:
train['soldYear'].value_counts()

soldYear
2022.00    24192
2023.00    23212
2024.00    22053
2021.00    20596
2020.00    10934
2019.00     7236
2018.00     6060
2017.00     5689
2016.00     4636
2015.00     3992
2014.00     2383
2013.00      847
2012.00      259
2010.00      160
2009.00      132
2011.00       99
2008.00       25
1901.00        5
1970.00        2
2002.00        1
2005.00        1
1910.00        1
Name: count, dtype: int64

In [87]:
# Not enough data for 2017 and earlier
train = train[train['soldYear'] > 2017]
test = test[test['soldYear'] > 2017]

### Sold price

In [88]:
# Removing unrealistic house prices
train = train[(train['soldPrice'] > 100000)]
train = train[train['soldPrice'] < 10000000]

test = test[(test['soldPrice'] > 100000)]
test = test[(test['soldPrice'] < 10000000)]

In [89]:
na_cols = ['suburb','bathrooms','bedrooms','parking',
           'soldYear','soldPrice']
# Removing missing rows for required variables soldPrice
train.dropna(subset=na_cols, inplace=True)
test.dropna(subset=na_cols, inplace=True)

In [90]:
print(train.shape)
print(test.shape)

(113529, 14)
(24227, 14)


# Reformatting 

In [91]:
train.shape

(113529, 14)

### Property type

In [92]:
train['propertyType'].value_counts()

propertyType
House                                                             94513
Apartment                                                         13187
Townhouse/Villa                                                    4265
Any                                                                1118
House,Any                                                           216
House,House                                                          43
Apartment,House,Townhouse/Villa,Townhouse/Villa                      24
House,Townhouse/Villa                                                23
Apartment,House                                                      21
Apartment,House,Townhouse/Villa                                      20
Apartment,Townhouse/Villa                                            20
Any,Any                                                              14
House,Land                                                           10
Apartment,House,Any,Townhouse/Villa,Land,Townhouse/

### Only possible entries:
- House
- Apartment
- townhouse??


In [93]:
train['propertyType'] = train['propertyType'].str.split(',')
test['propertyType'] = test['propertyType'].str.split(',')

def reformat_propertyTypes(row):
    # Valid property types
    types = ['House','Apartment']
    # This deals with NaN values
    if not isinstance(row, list):
        return 'Unknown'
    # Go through all property types
    for t in row:
        if t in types:
            # Return the first instance of a valid property type
            return t
    # Return 'Unkown' if no valid property types found
    return 'Unknown'
        
# Applying the function
train['propertyType'] = train['propertyType'].apply(reformat_propertyTypes)
test['propertyType'] = test['propertyType'].apply(reformat_propertyTypes)

In [94]:
# Changing the rural values from propertyTypes to House
train['propertyType'] = train['propertyType'].apply(lambda x: 'unit' if x.lower() =='apartment' else x)
test['propertyType'] = test['propertyType'].apply(lambda x: 'unit' if x.lower() =='apartment' else x)

# Removing unknown propertyTypes
train = train[(train['propertyType'] != 'Unknown')]
test = test[(test['propertyType'] != 'Unknown')]

In [95]:
train['propertyType'].value_counts()

propertyType
House    94819
unit     13309
Name: count, dtype: int64

In [96]:
train.shape

(108128, 14)

### Features

In [97]:
train['features'].value_counts()

features
['Ensuite']                                                                                                                                                                                                9399
['Air conditioning']                                                                                                                                                                                       1362
['Secure Parking']                                                                                                                                                                                         1210
['Study']                                                                                                                                                                                                   834
['Air conditioning', 'Built in wardrobes']                                                                                                                     

In [98]:
train['features'][3][0]

'['

We can see that each instance of the variables 'features' looks like a list but is actually a string

In [99]:
# Converting the 'string lists' into lists
# https://stackoverflow.com/questions/1894269/how-to-convert-string-representation-of-list-to-a-list
import ast

train['features'] = train['features'].apply(lambda x: ast.literal_eval(x) if pd.notnull(x) else [])
test['features'] = test['features'].apply(lambda x: ast.literal_eval(x) if pd.notnull(x) else [])

### Lowercasing categorical variables

In [100]:
train = train.map(lambda x: x.lower() if isinstance(x, str) else x)
test = test.map(lambda x: x.lower() if isinstance(x,str) else x)

In [101]:
# Lowercasing features variable
def lower_features(feature_list):
    f = []
    if len(feature_list) == 0:
        return feature_list
    else:
        for feature in feature_list:
            f.append(feature.lower())
    return f

train['features'] = train['features'].apply(lambda x: lower_features(x))
test['features'] = test['features'].apply(lambda x: lower_features(x))

# Missing values

In [102]:
train.isnull().sum()

streetNumber      527
street            490
suburb              0
postcode            0
bathrooms           0
bedrooms            0
parking             0
landArea        17324
latitude            2
longitude           2
features            0
propertyType        0
soldYear            0
soldPrice           0
dtype: int64

### Latitude and Longitude

We will be applying two methods in order to fill latitude and longitude
- Method 1 will be to use Nominatim to search for the address
- Method 2 will be to use a csv we made to merge and fill using the latitude and longitude of the suburb

In [103]:
# Initialise nominatim
ctx = ssl.create_default_context(cafile=certifi.where())
geopy.geocoders.options.default_ssl_context = ctx
nom = Nominatim(user_agent="tristan_scrape")


In [104]:
def format_address(house):
    parts = []
    # Check for street and street number, append if both are present
    if pd.notnull(house['street']) and pd.notnull(house['streetNumber']):
        parts.append(f'{house["streetNumber"]} {house["street"]}')
    elif pd.notnull(house['street']):
        parts.append(house['street'])
    
    # Check and append if suburb and postcode are present
    if pd.notnull(house['suburb']):
        parts.append(house['suburb'])
    if pd.notnull(house['postcode']):
        parts.append(f'{house["postcode"].astype(int)}')
    if parts:  # if there's at least one part present, append the state
        parts.append('Western Australia')
    # Join the parts with a comma and return the result
    print(parts)
    return ', '.join(parts) if parts else 'None'

In [105]:
def get_coords(address):
    # Searches Nominatim of the address
    location = nom.geocode(address)
    time.sleep(1)
    if location:
        return location.latitude, location.longitude
    return None, None

In [106]:

def coords_fill(df):
    # Getting index of data to fill
    index = df.loc[(df['latitude'].isnull()) & (df['suburb'].notnull())].index
    for idx in index:
        house = df.loc[idx]
        address = format_address(house)
        latitude, longitude = get_coords(address)
        # Assigning new latitude and longitude
        df.loc[idx, 'latitude'] = latitude
        df.loc[idx, 'longitude'] = longitude
    return df

In [107]:
train = coords_fill(train)
test = coords_fill(test)

['tooma close', 'lakelands', '6180', 'Western Australia']
['wintergreen crs', 'treeby', '6164', 'Western Australia']


In [108]:
suburbs_path = '/Users/tristangarcia/desktop/hp-pred/data/'
suburb_statistics = pd.read_csv(f'{suburbs_path}wa_suburb_statistics.csv')

suburb_statistics.head()

,suburb,state,postcode,latitude,longitude,marriedPercentage,ownerOccupierPercentage,population,renterPercentage,singlePercentage,mostCommonAgeBracket
0,gibson desert north,wa,872,-21.95,131.30,NaN,NaN,NaN,NaN,NaN,NaN
1,gibson desert south,wa,872,-24.95,125.98,NaN,NaN,NaN,NaN,NaN,NaN
2,irrunytju,wa,872,-26.06,128.93,NaN,NaN,NaN,NaN,NaN,NaN
3,kanpa,wa,872,-26.53,125.62,NaN,NaN,NaN,NaN,NaN,NaN
4,kiwirrkurra,wa,872,-23.28,126.95,NaN,NaN,NaN,NaN,NaN,NaN


In [109]:
coord_df = suburb_statistics[['suburb','latitude','longitude']]
print(coord_df.shape)
coord_df.head()

(1815, 3)


,suburb,latitude,longitude
0,gibson desert north,-21.95,131.30
1,gibson desert south,-24.95,125.98
2,irrunytju,-26.06,128.93
3,kanpa,-26.53,125.62
4,kiwirrkurra,-23.28,126.95


In [110]:
# Merging train with coord_df
train = pd.merge(train, coord_df, on=['suburb'], how='left', suffixes=('', '_from_coord'))
test = pd.merge(test, coord_df, on=['suburb'], how='left', suffixes=('', '_from_coord'))

# Filling missing Latitude and Longitude
train['latitude'] = train['latitude'].fillna(train['latitude_from_coord'])
train['longitude'] = train['longitude'].fillna(train['longitude_from_coord'])
test['latitude'] = test['latitude'].fillna(test['latitude_from_coord'])
test['longitude'] = test['longitude'].fillna(test['longitude_from_coord'])

# Dropping the extra columns from coord_df
train.drop(['latitude_from_coord', 'longitude_from_coord'], axis=1, inplace=True)
test.drop(['latitude_from_coord', 'longitude_from_coord'], axis=1, inplace=True)

train.isnull().sum()

streetNumber      527
street            490
suburb              0
postcode            0
bathrooms           0
bedrooms            0
parking             0
landArea        17324
latitude            0
longitude           0
features            0
propertyType        0
soldYear            0
soldPrice           0
dtype: int64

# KNN Imputation

In [111]:
from sklearn.neighbors import KNeighborsRegressor

### Land area
We will be using latitude and longitude to impute on landArea. We suspect that there is a difference in landArea for rural and non-rural areas. There also seems to be less recorded neighbours for rural areas. To account for this, the data will be split into rural and non-rural areas before imputation. Additionally, rural areas will find only 1 nearest neighbour will non-rural will find 3 nearest neighbours

In [112]:
train.reset_index(drop=True,inplace=True)
test.reset_index(drop=True,inplace=True)
# Seperating null and non-null 
x = train.loc[train['landArea'].notnull(), ['latitude','longitude','landArea']]
y = train.loc[train['landArea'].isnull(), ['latitude','longitude','landArea']]
y_test = test.loc[test['landArea'].isnull(), ['latitude','longitude','landArea']]

In [113]:
print(x.shape)
print(y.shape)

(90804, 3)
(17324, 3)


In [114]:
imputer = KNeighborsRegressor(n_neighbors=1)
# Fitting on the training set
imputer.fit(x[['latitude','longitude']], x['landArea'])
# Imputing on missing landArea 
predict = imputer.predict(y[['latitude', 'longitude']])
train.loc[y.index,'landArea'] = predict
# Imputing on missing landArea on validation set using training data
predict_test = imputer.predict(y_test[['latitude', 'longitude']])
test.loc[y_test.index,'landArea'] = predict_test

# Writing to file

In [115]:
print(train.shape)
print(test.shape)

(108128, 14)
(23079, 14)


In [116]:
train.to_pickle(f'{path}wa_train_clean.pkl')
test.to_pickle(f'{path}wa_validation_clean.pkl')

In [117]:
train.head(20)

,streetNumber,street,suburb,postcode,bathrooms,bedrooms,parking,landArea,latitude,longitude,features,propertyType,soldYear,soldPrice
0,18,townsend road,rockingham,6168.00,2,4,3,684.00,-32.29,115.73,[],house,2022.00,537500.00
1,1,mckivett crescent,leeming,6149.00,2,4,2,739.00,-32.08,115.85,[],house,2019.00,635000.00
2,72,oceanside promenade,mullaloo,6027.00,2,6,2,719.00,-31.78,115.73,"[air conditioning, balcony / deck, built in wardrobes, close to schools, close to shops, close to transport, garden, swimming pool, secure parking]",house,2023.00,2025000.00
3,19,raymond street,yokine,6060.00,1,2,1,62.00,-31.90,115.84,[],unit,2022.00,220000.00
4,1,eucumbene crescent,joondalup,6027.00,1,3,1,651.00,-31.72,115.77,"[life style, value]",house,2021.00,421000.00
5,165b,bateman road,mount pleasant,6153.00,3,4,2,506.00,-32.04,115.85,[],house,2024.00,2125000.00
6,46,thorburn avenue,beechboro,6063.00,1,3,1,696.00,-31.86,115.95,[],house,2023.00,525000.00
7,135,challis road,seville grove,6112.00,2,3,1,263.00,-32.13,116.00,"[air conditioning, ensuite, alarm system, pets allowed, internal laundry, close to shops, close to transport]",house,2023.00,330000.00
8,25,craike way,green head,6514.00,2,3,7,765.00,-30.07,114.97,"[secure parking, ensuite]",house,2022.00,430000.00
9,10a,simeon close,kalamunda,6076.00,2,4,3,4373.00,-31.96,116.03,"[air conditioning, internal laundry, separate dining room, dishwasher, garden / courtyard]",house,2020.00,1050000.00


In [118]:
train.isnull().sum()

streetNumber    527
street          490
suburb            0
postcode          0
bathrooms         0
bedrooms          0
parking           0
landArea          0
latitude          0
longitude         0
features          0
propertyType      0
soldYear          0
soldPrice         0
dtype: int64